# Evaluate Yolov11 with RAAD
Training Yolo11n Model and after Test with RAAD

In [1]:
!pip install -U ultralytics wandb
!pip install dotenv
!pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.9 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.142
    Uninstalling ultralytics-8.3.142:
      Successfully uninstalled ultralytics-8.3.142


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import wandb
from ultralytics import YOLO
from tqdm.notebook import tqdm
from shapely.geometry import box, MultiPolygon
import yaml
import torch
import pandas as pd
from dotenv import load_dotenv
import logging

In [3]:
# Enable W&B logging for Ultralytics
!yolo settings wandb=True

JSONDict("/home/jovyan/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/jovyan/DSPRO2/M-AI-ZE-Maize-diseases-detection/notebooks/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "8a115bbf5049f0fe55cf2ccd8be54ca8bfded6b963fd272724a959bb525556d2",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": true,
  "vscode_msg": true,
  "openvino_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [4]:
# Load the .env file
load_dotenv()

# Get and print the WANDB_API_KEY
wandb_api_key = os.getenv("WANDB_API_KEY")
print(f"WANDB_API_KEY: [{wandb_api_key[:4]}...]")

WANDB_API_KEY: [69ca...]


In [5]:
wandb.login(key=wandb_api_key)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: rueedi-tobias (rueedi-tobias-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Get Data

In [6]:
base_path = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits"
splits = ["SID01", "SID02", "SID03"]
test_csv_path = base_path + "/SID01/labels/test/bboxes_test.csv"

In [7]:
for split in splits:
    dataset_path = os.path.join(base_path, split)
    yaml_path = os.path.join(dataset_path, "data.yaml")
    
    print(f"Check Split: {split}")
    print(f"  Dataset Path: {dataset_path} - Existiert: {os.path.exists(dataset_path)}")
    print(f"  YAML Path: {yaml_path} - Existiert: {os.path.exists(yaml_path)}")

    if os.path.exists(yaml_path):
        with open(yaml_path, 'r') as f:
            data_yaml = yaml.safe_load(f)
            print(f"  YAML Inhalt: {data_yaml.keys()}")
            print(f"  Train: {data_yaml.get('train')}")
            print(f"  Val: {data_yaml.get('val')}")
            print(f"  Test: {data_yaml.get('test')}")
            print(f"  NC: {data_yaml.get('nc')}")
            print(f"  Names: {data_yaml.get('names')}")
    print()

Check Split: SID01
  Dataset Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01 - Existiert: True
  YAML Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml - Existiert: True
  YAML Inhalt: dict_keys(['path', 'train', 'val', 'test', 'names'])
  Train: images/train
  Val: images/val
  Test: images/test
  NC: None
  Names: {0: 'lesion'}

Check Split: SID02
  Dataset Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID02 - Existiert: True
  YAML Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID02/data.yaml - Existiert: False

Check Split: SID03
  Dataset Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID03 - Existiert: True
  YAML Path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID03/data.yaml - Existiert: False



## RAAD implementation


In [8]:
def calculate_raad(pred_boxes, true_boxes, img_width=640, img_height=640, epsilon=1e-6, normalize=True):
    """
    Calculate RAAD (Relative Affected Area Difference) metric
    
    Args:
        pred_boxes: List of predicted boxes in format [x1, y1, x2, y2]
        true_boxes: List of ground truth boxes in format [x1, y1, x2, y2]
        img_width: Width of the image
        img_height: Height of the image
        epsilon: Small value to avoid division by zero
        normalize: Whether to normalize coordinates from 0-1 to image dimensions
        
    Returns:
        RAAD value (lower is better), Area of predictions, Area of ground truth
    """
    if len(pred_boxes) == 0 and len(true_boxes) == 0:
        return 0.0, 0.0, 0.0
    if len(pred_boxes) == 0:
        return 1.0, 0.0, sum([(box[2]-box[0])*(box[3]-box[1]) for box in true_boxes])
    if len(true_boxes) == 0:
        return 1.0, sum([(box[2]-box[0])*(box[3]-box[1]) for box in pred_boxes]), 0.0
    
    if normalize:
        pred_boxes = [[box[0]*img_width, box[1]*img_height, 
                      box[2]*img_width, box[3]*img_height] for box in pred_boxes]
        true_boxes = [[box[0]*img_width, box[1]*img_height, 
                      box[2]*img_width, box[3]*img_height] for box in true_boxes]
    
    pred_polygons = [box(b[0], b[1], b[2], b[3]) for b in pred_boxes]
    true_polygons = [box(b[0], b[1], b[2], b[3]) for b in true_boxes]
    
    if pred_polygons:
        pred_multipolygon = MultiPolygon(pred_polygons).buffer(0)
    else:
        pred_multipolygon = MultiPolygon([])
        
    if true_polygons:
        true_multipolygon = MultiPolygon(true_polygons).buffer(0)
    else:
        true_multipolygon = MultiPolygon([])
    
    pred_area = pred_multipolygon.area
    true_area = true_multipolygon.area
    
    area_diff = abs(pred_area - true_area)
    raad = area_diff / max(true_area, epsilon)
    
    return raad, pred_area, true_area

In [9]:
def test_raad_calculation():
    pred_boxes1 = [[100, 100, 200, 200]]
    true_boxes1 = [[100, 100, 200, 200]]
    
    pred_boxes2 = [[100, 100, 200, 200]]
    true_boxes2 = [[300, 300, 400, 400]]
    
    pred_boxes3 = [[100, 100, 300, 300]]
    true_boxes3 = [[200, 200, 400, 400]]

    pred_boxes4 = [[100, 100, 400, 400]]
    true_boxes4 = [[150, 150, 300, 300]]
    
    pred_boxes5 = [[100, 100, 300, 300], [250, 250, 450, 450]]
    true_boxes5 = [[150, 150, 350, 350]]
    
    examples = [
        (pred_boxes1, true_boxes1, "Identical Boxes"),
        (pred_boxes2, true_boxes2, "No Overlaps"),
        (pred_boxes3, true_boxes3, "Partly Overlapping"),
        (pred_boxes4, true_boxes4, "Different Boxes"),
        (pred_boxes5, true_boxes5, "Multiple Boxes with Overlapps")
    ]
    
    for pred_boxes, true_boxes, desc in examples:
        raad, pred_area, true_area = calculate_raad(pred_boxes, true_boxes, normalize=False)
        print(f"{desc}:")
        print(f"  RAAD: {raad:.4f}")
        print(f"  Pred Area: {pred_area:.1f}, True Area: {true_area:.1f}")
        print(f"  Area Diff: {abs(pred_area - true_area):.1f}")
        print()

In [10]:
test_raad_calculation()

Identical Boxes:
  RAAD: 0.0000
  Pred Area: 10000.0, True Area: 10000.0
  Area Diff: 0.0

No Overlaps:
  RAAD: 0.0000
  Pred Area: 10000.0, True Area: 10000.0
  Area Diff: 0.0

Partly Overlapping:
  RAAD: 0.0000
  Pred Area: 40000.0, True Area: 40000.0
  Area Diff: 0.0

Different Boxes:
  RAAD: 3.0000
  Pred Area: 90000.0, True Area: 22500.0
  Area Diff: 67500.0

Multiple Boxes with Overlapps:
  RAAD: 0.9375
  Pred Area: 77500.0, True Area: 40000.0
  Area Diff: 37500.0



## Training SID01 with RAAD

In [11]:
def train_model(model_type="yolo11n.pt", split="SID01", epochs=10, batch=16, lr=0.01):
    """
    Train a YOLO model with RAAD metric tracking
    
    This version properly checks for the model file after training and handles paths correctly.
    """
    dataset_yaml = f"{base_path}/{split}/data.yaml"
    
    model_name = model_type.split('.')[0]
    
    run_name = f"{model_name}_{split}_raad_e{epochs}"
    
    try:
        model = YOLO(model_type)
        
        results = model.train(
            data=dataset_yaml,
            epochs=epochs,
            imgsz=640,
            lr0=lr,
            batch=batch,
            name=run_name,
            project="maize_detection",
            exist_ok=True
        )
        
        potential_paths = [
            f"runs/detect/{run_name}/weights/best.pt",
            f"maize_detection/{run_name}/weights/best.pt",
            f"runs/train/{run_name}/weights/best.pt",
            f"maize_detection/{run_name}/weights/last.pt",
            f"runs/detect/{run_name}/weights/last.pt"
        ]
        
        weights_path = None
        for path in potential_paths:
            if os.path.exists(path):
                weights_path = path
                print(f"Found model weights at: {weights_path}")
                break
        
        if weights_path is None:
            print("Warning: Could not find model weights file. Here are the directories in runs/:")
            if os.path.exists("runs"):
                print(os.listdir("runs"))
                if os.path.exists("runs/detect"):
                    print("Contents of runs/detect/:")
                    print(os.listdir("runs/detect"))
                    detect_dirs = os.listdir("runs/detect")
                    if detect_dirs:
                        first_dir = os.path.join("runs/detect", detect_dirs[0])
                        print(f"Contents of {first_dir}:")
                        print(os.listdir(first_dir))
                        if os.path.exists(os.path.join(first_dir, "weights")):
                            print(f"Contents of {first_dir}/weights:")
                            print(os.listdir(os.path.join(first_dir, "weights")))
            
            print("Using original model as fallback since trained weights not found")
            return model_type  # Return the original model path
        
        if wandb.run is not None:
            wandb.finish()
        
        wandb_config = {
            "model": model_type,
            "epochs": epochs,
            "batch": batch,
            "lr": lr,
            "split": split,
            "weights_path": weights_path
        }
        
        additional_run = wandb.init(
            project=f"maize_detection_{split}_additional", 
            name=f"{model_name}_{split}_additional_metrics_e{epochs}",
            config=wandb_config,
            reinit=True
        )
        
        try:
            metrics_table = wandb.Table(columns=["model", "split", "mAP50", "mAP50-95", "weights_path"])
            
            map50 = results.results_dict.get('metrics/mAP50', 0)
            map50_95 = results.results_dict.get('metrics/mAP50-95', 0)
            
            metrics_table.add_data(model_type, split, map50, map50_95, weights_path)
            
            log_dict = {"metrics_comparison": metrics_table}
            
            try:
                val_path = os.path.join(base_path, split, "images", "val")
                if os.path.exists(val_path):
                    val_images = [f for f in os.listdir(val_path) if f.endswith(('.jpg', '.png'))][:5]  # Nimm die ersten 5 Bilder
                    
                    for i, img_file in enumerate(val_images):
                        img_path = os.path.join(val_path, img_file)
                        prediction_results = model.predict(img_path, conf=0.25)
                        
                        img = cv2.imread(img_path)
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        
                        for r in prediction_results:
                            boxes = r.boxes
                            for box in boxes:
                                x1, y1, x2, y2 = box.xyxy[0].tolist()
                                conf = box.conf[0].item()
                                
                                cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                                cv2.putText(img, f"{conf:.2f}", (int(x1), int(y1) - 10), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        
                        log_dict[f"validation_image_{i}"] = wandb.Image(img, caption=img_file)
                        
            except Exception as e:
                print(f"Fehler beim Visualisieren der Validierungsbilder: {e}")
                
            # Make a single log call with all our data
            additional_run.log(log_dict)
            
        finally:
            additional_run.finish()
        
        return weights_path
    
    except Exception as e:
        print(f"Error during training: {e}")
        traceback.print_exc()
        return model_type

## Evaluate RAAD on Testdata

In [12]:
def suppress_yolo_logging():
    """Suppress the YOLO logging temporarily."""
    logger = logging.getLogger("ultralytics")
    original_level = logger.level
    logger.setLevel(logging.ERROR)  # Zeige nur Fehler an
    return logger, original_level

def restore_yolo_logging(logger, original_level):
    """Restore the YOLO logging to its original level."""
    logger.setLevel(original_level)


In [13]:
def load_bounding_boxes_from_csv(csv_path):
    """
    Load bounding boxes from a single CSV file.
    
    Args:
        csv_path (str): Path to the CSV file.
        
    Returns:
        dict: Dictionary where keys are image names and values are lists of bounding boxes.
    """
    df = pd.read_csv(csv_path, header=None, skiprows=1)
    bounding_boxes = {}

    for _, row in df.iterrows():
        img_name = row[0]
        x1, y1, x2, y2 = map(int, [row[1], row[2], row[3], row[4]])

        if img_name not in bounding_boxes:
            bounding_boxes[img_name] = []
        bounding_boxes[img_name].append((x1, y1, x2, y2))

    print(f"Loaded bounding boxes for {len(bounding_boxes)} images.")
    return bounding_boxes


In [14]:
def evaluate_test_raad(model_path, bounding_boxes, split="SID01"):
    """
    Evaluate a trained model on test data with RAAD metric.
    
    Args:
        model_path (str): Path to the YOLO model file.
        bounding_boxes (dict): Dictionary with image names as keys and box lists as values.
        split (str): Data split name.
        
    Returns:
        DataFrame: Evaluation results with RAAD values.
    """
    model_name = os.path.basename(model_path).split('.')[0]
    
    run = wandb.init(project="maize_detection_test", 
                    name=f"test_evaluation_{model_name}_{split}",
                    config={"model_path": model_path, "split": split})
    
    try:
        test_images_dir = f"{base_path}/{split}/images/test"
        
        model = YOLO(model_path)
        
        results = {
            'image': [],
            'raad': [],
            'pred_area': [],
            'true_area': [],
            'prediction_count': [],
            'truth_count': []
        }
        
        test_images = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.png'))]
        
        for img_file in tqdm(test_images, desc="Evaluating test images"):
            img_path = os.path.join(test_images_dir, img_file)
            
            if img_file not in bounding_boxes:
                print(f"No ground truth for image: {img_file}")
                continue
            
            img = cv2.imread(img_path)
            img_height, img_width = img.shape[:2]
            
            logger, original_level = suppress_yolo_logging()
            try:
                predictions = model.predict(img_path, save=False)
            finally:
                restore_yolo_logging(logger, original_level)

            
            pred_boxes = [
                [int(box[0]), int(box[1]), int(box[2]), int(box[3])]
                for r in predictions for box in r.boxes.xyxy.cpu().numpy()
            ]

            logger, original_level = suppress_yolo_logging()
            
            true_boxes = bounding_boxes[img_file]
            
            raad, pred_area, true_area = calculate_raad(pred_boxes, true_boxes, img_width, img_height, normalize=False)
            
            results['image'].append(img_file)
            results['raad'].append(raad)
            results['pred_area'].append(pred_area)
            results['true_area'].append(true_area)
            results['prediction_count'].append(len(pred_boxes))
            results['truth_count'].append(len(true_boxes))
        
        results_df = pd.DataFrame(results)
        avg_raad = results_df['raad'].mean()
        print(f"Average RAAD on test set: {avg_raad:.4f}")
        run.log({"test/avg_raad": avg_raad})
        run.finish()
        
        return results_df, avg_raad
    
    except Exception as e:
        print(f"Error during RAAD evaluation: {e}")
        run.finish()
        return None, None


In [17]:
weights_path = train_model(model_type="yolo11n.pt", split="SID01", epochs=1, batch=16, lr=0.01)
print(f"Trained Modell saved at: {weights_path}")

Ultralytics 8.3.139 🚀 Python-3.12.8 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_SID01_raad_e1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1622.2±697.2 MB/s, size: 59.1 KB)


train: Scanning /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/train.cache... 10858 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10858/10858 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1735.3±896.9 MB/s, size: 56.1 KB)


val: Scanning /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/val.cache... 1357 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1357/1357 [00:00<?, ?it/s]


...

lr/pg0,▁
lr/pg1,▁
lr/pg2,▁
metrics/mAP50(B),▁
metrics/mAP50-95(B),▁
metrics/precision(B),▁
metrics/recall(B),▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▁


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Found model weights at: maize_detection/yolo11n_SID01_raad_e1/weights/best.pt



image 1/1 /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val/DSC02226_2.jpg: 640x640 4 lesions, 6.6ms
Speed: 1.4ms preprocess, 6.6ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val/J_170911_134911.jpg: 640x640 2 lesions, 7.2ms
Speed: 1.3ms preprocess, 7.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val/J_170825_145123.jpg: 640x640 (no detections), 7.1ms
Speed: 1.3ms preprocess, 7.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val/DSC03294_0.jpg: 640x640 4 lesions, 7.2ms
Speed: 1.3ms preprocess, 7.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val/J_170830_154632.jpg: 640x640 (no detections), 

Trained Modell saved at: maize_detection/yolo11n_SID01_raad_e1/weights/best.pt


In [18]:
bounding_boxes = load_bounding_boxes_from_csv(test_csv_path)
test_results, avg_raad = evaluate_test_raad(weights_path, bounding_boxes, split="SID01")

Loaded bounding boxes for 1358 images.


Evaluating test images:   0%|          | 0/1358 [00:00<?, ?it/s]

Average RAAD on test set: 0.9170


test/avg_raad,▁
test/avg_raad,0.917


## Multiple Models:
Work in Progress

In [1]:
def train_multiple_models(model_types, splits, epochs_list, batch=16, lr=0.01):
    """
    Train multiple YOLO models with various configurations
    
    Args:
        model_types: List of model types (e.g. ["yolo11n.pt", "yolo11l.pt"])
        splits: List of splits to train on (e.g. ["SID01", "SID02"])
        epochs_list: List of epoch numbers to train for each model
        batch: Batch size
        lr: Learning rate
    
    Returns:
        Dictionary with training results
    """
    results = {}
    
    for model_type in model_types:
        for split in splits:
            for epochs in epochs_list:
                print(f"\n=== Training {model_type} on {split} for {epochs} epochs ===")
                
                config_key = f"{model_type.split('.')[0]}_{split}_e{epochs}"
                
                try:
                    weights_path = train_model(model_type=model_type, split=split, epochs=epochs, batch=batch, lr=lr)
                    
                    print(f"\n=== Evaluating {model_type} on {split} test set ===")
                    _, avg_raad = evaluate_test_raad(weights_path, split=split)
                    
                    results[config_key] = {
                        "model_type": model_type,
                        "split": split,
                        "epochs": epochs,
                        "weights_path": weights_path,
                        "avg_raad": avg_raad
                    }
                    
                    print(f"Completed training and evaluation for {config_key}")
                    print(f"Model saved at: {weights_path}")
                    print(f"Average RAAD: {avg_raad:.4f}")
                    
                except Exception as e:
                    print(f"Error training {config_key}: {e}")
                    results[config_key] = {
                        "model_type": model_type,
                        "split": split,
                        "epochs": epochs,
                        "error": str(e)
                    }
    
    return results


## Training SID01 with Multiple Models

In [47]:
def main():
    wandb.login()
    
    model_types = ["yolo11n.pt", "yolo11l.pt", "yolo11m.pt"]
    splits = ["SID01"]
    #splits = ["SID01", "SID02", "SID03"]
    epochs_list = [40]
    batch = [16, 32]
    
    print("=== Training multiple YOLO models ===")
    results = train_multiple_models(
        model_types=model_types,
        splits=splits,
        epochs_list=epochs_list,
        batch=16,
        lr=0.01
    )
    
    print("\n=== Training Results Summary ===")
    for config_key, result in results.items():
        if "error" in result:
            print(f"{config_key}: Failed - {result['error']}")
        else:
            print(f"{config_key}: RAAD = {result['avg_raad']:.4f}, Model: {result['weights_path']}")
    
    print("\n=== Training Complete ===")

if __name__ == "__main__":
    main()

=== Training multiple YOLO models ===

=== Training yolo11n.pt on SID01 for 10 epochs ===


Ultralytics 8.3.139 🚀 Python-3.12.8 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_SID01_raad_e10, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

train: Scanning /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/train.cache... 10858 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10858/10858 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1270.9±807.5 MB/s, size: 56.1 KB)


val: Scanning /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/val.cache... 1357 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1357/1357 [00:00<?, ?it/s]


Plotting labels to maize_detection/yolo11n_SID01_raad_e10/labels.jpg... 
WARNING ⚠️ 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to maize_detection/yolo11n_SID01_raad_e10
Starting training for 10 epochs...
Closing dataloader mosaic


Exception in thread Thread-38 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 59, in _pin_memory_loop
    do_one_step()
  File "/opt/conda/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 35, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^Exception in thread ^Thread-39 (_pin_memory_loop):
Traceback (most recent


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size



  File "/opt/conda/lib/python3.12/multiprocessing/resource_sharer.py", line 86, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/multiprocessing/connection.py", line 519, in Client
    c = SocketClient(address)
        ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/multiprocessing/connection.py", line 647, in SocketClient
    s.connect(address)
FileNotFoundError: [Errno 2] No such file or directory
      with _resource_sharer.get_connection(self._id) as conn:  110        640:   1%|          | 7/679 [00:02<02:56,  3.81it/s]
         ^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/multiprocessing/resource_sharer.py", line 86, in get_connection
    

KeyboardInterrupt: 

In [ ]:
def train_all_splits(model_type="yolo11n.pt", epochs=10):
    """
    Train on all splits and compare results
    """
    splits = ["SID01", "SID02", "SID03"]
    base_path = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits"
    
    # Summary metrics
    summary_table = wandb.Table(columns=["model", "split", "mAP50", "RAAD", "training_time"])
    
    for split in splits:
        # SID03 uses k-fold
        if split == "SID03":
            # Implement simple 2-fold cross-validation
            # For SID03 which doesn't have validation set
            train_folders = [
                f"{base_path}/{split}/labels/train/boom",
                f"{base_path}/{split}/labels/train/drone",
                f"{base_path}/{split}/labels/train/handheld"
            ]
            
            # Get all training files from all folders
            all_files = []
            for folder in train_folders:
                files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.txt')]
                all_files.extend(files)
            
            # Split into two folds
            import random
            random.shuffle(all_files)
            fold_size = len(all_files) // 2
            fold1 = all_files[:fold_size]
            fold2 = all_files[fold_size:]
            
            # Create temporary data yamls for each fold
            fold1_yaml = f"{base_path}/{split}/data_fold1.yaml"
            fold2_yaml = f"{base_path}/{split}/data_fold2.yaml"
            
            # Create yamls (simplified, you'll need to adapt this)
            with open(f"{base_path}/{split}/data.yaml", 'r') as f:
                data_yaml = f.read()
            
            # Save fold yaml files
            # This is simplified - you'll need to adapt this to create proper yamls
            
            # Train on fold 1, validate on fold 2
            run = wandb.init(project=f"maize_detection_splits", 
                            name=f"{model_type.split('.')[0]}_{split}_fold1",
                            config={
                                "model": model_type,
                                "epochs": epochs,
                                "split": f"{split}_fold1"
                            })
            
            model = YOLO(model_type)
            start_time = time.time()
            
            # Train (simplified)
            results = model.train(
                data=fold1_yaml,  # You'll need to create this
                epochs=epochs,
                imgsz=640,
                name=f"{model_type.split('.')[0]}_{split}_fold1",
                project="maize_detection_splits",
                exist_ok=True
            )
            
            training_time = time.time() - start_time
            
            # Add to summary
            map50 = results.results_dict.get('metrics/mAP50', 0)
            raad_value = results.results_dict.get('raad', 0)
            
            summary_table.add_data(model_type, f"{split}_fold1", map50, raad_value, training_time)
            
            run.finish()
            
            # Repeat for fold 2 (code omitted for brevity)
            
        else:
            # Regular training for SID01 and SID02
            dataset_yaml = f"{base_path}/{split}/data.yaml"
            
            run = wandb.init(project=f"maize_detection_splits", 
                            name=f"{model_type.split('.')[0]}_{split}",
                            config={
                                "model": model_type,
                                "epochs": epochs,
                                "split": split
                            })
            
            model = YOLO(model_type)
            start_time = time.time()
            
            results = model.train(
                data=dataset_yaml,
                epochs=epochs,
                imgsz=640,
                name=f"{model_type.split('.')[0]}_{split}",
                project="maize_detection_splits",
                exist_ok=True
            )
            
            training_time = time.time() - start_time
            
            # Add to summary
            map50 = results.results_dict.get('metrics/mAP50', 0)
            raad_value = results.results_dict.get('raad', 0)
            
            summary_table.add_data(model_type, split, map50, raad_value, training_time)
            
            run.finish()
    
    # Create final summary run
    summary_run = wandb.init(project="maize_detection_summary", name=f"{model_type.split('.')[0]}_summary")
    wandb.log({"split_comparison": summary_table})
    summary_run.finish()

In [ ]:
import wandb
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

def analyze_results():
    """
    Analyze and visualize training results across splits and models
    """
    # Initialize wandb
    run = wandb.init(project="maize_detection_analysis", name="final_analysis")
    
    # Load experiment results from wandb API
    api = wandb.Api()
    
    # Get runs from different projects
    baseline_runs = api.runs("maize_detection_baseline")
    split_runs = api.runs("maize_detection_splits")
    
    # Extract metrics into pandas dataframes
    baseline_df = pd.DataFrame([
        {
            "model": run.config.get("model", "unknown"),
            "split": "SID01",  # Baseline only used SID01
            "mAP50": run.summary.get("metrics/mAP50", 0),
            "raad": run.summary.get("raad", 0),
            "training_time": run.summary.get("_runtime", 0) / 60  # minutes
        }
        for run in baseline_runs
    ])
    
    split_df = pd.DataFrame([
        {
            "model": run.config.get("model", "unknown"),
            "split": run.config.get("split", "unknown"),
            "mAP50": run.summary.get("metrics/mAP50", 0),
            "raad": run.summary.get("raad", 0),
            "training_time": run.summary.get("_runtime", 0) / 60  # minutes
        }
        for run in split_runs
    ])
    
    # Combine dataframes
    all_df = pd.concat([baseline_df, split_df])
    
    # Create visualizations
    
    # 1. mAP50 vs RAAD scatter plot
    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=all_df, x="mAP50", y="raad", hue="split", size="model", sizes=(100, 200))
    plt.title("mAP50 vs RAAD by Split and Model")
    plt.xlabel("mAP50")
    plt.ylabel("RAAD (lower is better)")
    plt.savefig("map_vs_raad.png")
    wandb.log({"map_vs_raad": wandb.Image("map_vs_raad.png")})
    
    # 2. Model performance by split (bar chart)
    plt.figure(figsize=(14, 10))
    
    # Plot for mAP50
    plt.subplot(2, 1, 1)
    sns.barplot(data=all_df, x="model", y="mAP50", hue="split")
    plt.title("mAP50 by Model and Split")
    plt.xticks(rotation=45)
    
    # Plot for RAAD
    plt.subplot(2, 1, 2)
    sns.barplot(data=all_df, x="model", y="raad", hue="split")
    plt.title("RAAD by Model and Split (lower is better)")
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig("model_performance.png")
    wandb.log({"model_performance": wandb.Image("model_performance.png")})
    
    # 3. Create summary table
    summary_table = wandb.Table(dataframe=all_df)
    wandb.log({"summary_results": summary_table})
    
    # 4. Key findings
    best_map_model = all_df.loc[all_df["mAP50"].idxmax()]
    best_raad_model = all_df.loc[all_df["raad"].idxmin()]  # Lower RAAD is better
    
    findings = f"""
    # Key Findings
    
    ## Best Model by mAP50
    - Model: {best_map_model['model']}
    - Split: {best_map_model['split']}
    - mAP50: {best_map_model['mAP50']:.4f}
    - RAAD: {best_map_model['raad']:.4f}
    
    ## Best Model by RAAD
    - Model: {best_raad_model['model']}
    - Split: {best_raad_model['split']}
    - mAP50: {best_raad_model['mAP50']:.4f}
    - RAAD: {best_raad_model['raad']:.4f}
    
    ## Split Comparisons
    - SID01 (Standard Split): Average mAP50 = {all_df[all_df['split'] == 'SID01']['mAP50'].mean():.4f}
    - SID02 (Device-specific): Average mAP50 = {all_df[all_df['split'] == 'SID02']['mAP50'].mean():.4f}
    - SID03 (K-fold): Average mAP50 = {all_df[all_df['split'].str.contains('SID03')]['mAP50'].mean():.4f}
    
    ## RAAD vs mAP50
    - Correlation between RAAD and mAP50: {all_df['raad'].corr(all_df['mAP50']):.4f}
    """
    
    with open("findings.md", "w") as f:
        f.write(findings)
    
    wandb.save("findings.md")
    
    run.finish()